# 5.1 スクレイピング

## 5.1.2 スクレイピング環境の準備

```
(pydataenv) $ pip install requests==2.19.1
(pydataenv) $ pip install beautifulsoup4==4.6.0
```

* Requests: http://docs.python-requests.org/
* BeautifulSoup4: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

## 5.1.3 Webページをダウンロード

* CodeZine https://codezine.jp/

In [1]:
import requests

r = requests.get('https://codezine.jp')  # URLにアクセスする
print(type(r))
print(r.status_code)  # ステータスコードを確認

<class 'requests.models.Response'>
200


In [2]:
text = r.text  # HTMLのソースコードを取得する
for line in text.split('\n'):
    if'<title>' in line or '<h1>' in line:
        print(line.strip())

<title>CodeZine（コードジン）</title>
<h1><a href="/"><img src="/lib/img/cmn/cmn-header-logo.png" alt="CodeZine（コードジン）" ></a></h1>


## 5.1.4 Webページから要素を抜き出す

In [3]:
from bs4 import BeautifulSoup

# HTMLを解析したオブジェクトを生成
soup = BeautifulSoup(text, 'html.parser')
print(soup.title)  # <title> タグの情報を取得
print(soup.h1)  # <h1> タグの情報を取得
# h1タグの中のaタグの中のimgタグの中のalt属性
print(soup.h1.a.img['alt'])

<title>CodeZine（コードジン）</title>
<h1><a href="/"><img alt="CodeZine（コードジン）" src="/lib/img/cmn/cmn-header-logo.png"/></a></h1>
CodeZine（コードジン）


In [4]:
atags = soup.find_all('a')  # すべてのaタグを取得
print('aタグの数:', len(atags))  # aタグの数を取得
for atag in atags[:5]:
    print('タイトル:', atag.text)  # aタグのテキストを取得
    print('リンク:', atag['href'])  # aタグのリンクを取得

aタグの数: 163
タイトル: このページの本文へ移動
リンク: #contents
タイトル: 企業IT
リンク: https://enterprisezine.jp/
タイトル: 開発
リンク: https://codezine.jp/
タイトル: データベース
リンク: https://enterprisezine.jp/dbonline/
タイトル: セキュリティ
リンク: https://enterprisezine.jp/securityonline/


## 5.1.5 記事の一覧を抜き出す

* CodeZine: https://codezine.jp/article/tag/223

In [5]:
from datetime import datetime

import requests
from bs4 import BeautifulSoup

r = requests.get('https://codezine.jp/article/tag/223')
soup = BeautifulSoup(r.text, 'html.parser')

articles = []  # 各記事の情報を格納するリスト

# CSSセレクターで <ul class="catList"><li> を取得
lis = soup.select('ul.catList > li')
for li in lis:
    # 日付の文字列を取得
    day = li.find('div', class_='day').text.strip()
    # 日付をdatetimeに変換
    published = datetime.strptime(day, '%Y/%m/%d')
    h2_tag = li.find('h2')  # h2タグを取得
    title = h2_tag.text  # タイトルを取得
    url = h2_tag.a['href']  # URLを取得
    
    tag_list = li.select('ul.tag > li')  # タグのli要素を取得
    # タグのリストを生成
    tags = [tag.text.strip() for tag in tag_list]
    
    article = {
        'publisthed': published,
        'title': title,
        'url': url,
        'tags': tags
    }
    articles.append(article)

In [6]:
articles[:3]

[{'publisthed': datetime.datetime(2018, 9, 6, 0, 0),
  'title': 'Pythonでデータと向き合いながら価値を見出す「探索的データ解析」に挑戦しよう',
  'url': '/article/detail/11043',
  'tags': ['Python']},
 {'publisthed': datetime.datetime(2018, 9, 1, 0, 0),
  'title': '「PyCon JP 2018 ひろがるPython」登壇者座談会 ～わたしのPythonのひろげかた～',
  'url': '/article/detail/11031',
  'tags': ['Python', 'イベントレポート']},
 {'publisthed': datetime.datetime(2018, 7, 5, 0, 0),
  'title': 'クイズ王たちを凌駕する早押しクイズAIはこう作る～PyData.Tokyo Meetup #18イベントレポート',
  'url': '/article/detail/10865',
  'tags': ['Python', 'レポート']}]

In [7]:
import pandas as pd

df = pd.DataFrame(articles)  # 辞書をDataFrameに変換

In [8]:
df

,publisthed,tags,title,url
0,2018-09-06,[Python],Pythonでデータと向き合いながら価値を見出す「探索的データ解析」に挑戦しよう,/article/detail/11043
1,2018-09-01,"[Python, イベントレポート]",「PyCon JP 2018 ひろがるPython」登壇者座談会 ～わたしのPythonのひ...,/article/detail/11031
2,2018-07-05,"[Python, レポート]",クイズ王たちを凌駕する早押しクイズAIはこう作る～PyData.Tokyo Meetup #...,/article/detail/10865
3,2018-07-02,"[Python, データ分析]",本格的なPythonデータ解析環境を手軽に！ 「Jupyter Notebook」の導入から...,/article/detail/10917
4,2018-05-17,"[Python, インタビュー, TensorFlow, Keras]",目指したのは気軽に深層学習を試せる本 『TensorFlow開発入門』著者陣が語る,/article/detail/10805
5,2018-04-20,"[Python, インタビュー, 統計学]",データを扱う開発者なら統計学の知識は大きな強みに～Pythonを使って基礎から学ぶ,/article/detail/10765
6,2018-04-13,"[Python, 統計学]",データ分析に不可欠の統計学をPythonで学べる！『Pythonで学ぶあたらしい統計学の教科...,/article/detail/10734
7,2018-01-29,"[Python, インタビュー, 機械学習]",機械学習の理解を数学で挫折してしまった人のために――『あたらしい機械学習の教科書』伊藤真さんに訊く,/article/detail/10631
8,2018-01-17,"[Python, 機械学習]",機械学習を始めたい！ まずは『Pythonで動かして学ぶ！ あたらしい機械学習の教科書』で感...,/article/detail/10583
9,2017-11-10,"[Python, イベントレポート]",「技術カンファレンスへの参加の秘訣は臆せず飛び込むこと」――PyCon JP 2017発表者...,/article/detail/10507


## 5.1.6 スクレイピングで気をつけること

* Robots.txtの仕様: https://developers.google.com/search/reference/robots_txt?hl=ja

## 5.1.7 次のステップ

* Selenium: https://www.seleniumhq.org/
* Scrapy: https://scrapy.org/